# Week 5: Directed Area
**Target**: 

In [1]:
# Packages
import numpy as np
import random
import matplotlib.pyplot as plt

# Self-defined functions
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from util import util

# Visualization
import pyvista as pv
from pyvista import examples

In [2]:
def line_equation(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    if x1 == x2:
        return np.array([1, 0, -x1])
    else:
        return np.array([-(y2-y1), x2-x1, x1*(y2-y1)-y1*(x2-x1)])

# 测试函数
point1 = np.array([1, 2])
point2 = np.array([3, 4])
a, b, c = line_equation(point1, point2)

print(f"The line equation is: {a}x + {b}y + {c} = 0")

The line equation is: -2x + 2y + -2 = 0


In [3]:
def line_intersection(l1, l2):
    A1, B1, C1 = l1
    A2, B2, C2 = l2

    coeff_matrix = np.array([[A1, B1], [A2, B2]])
    result_vector = np.array([-C1, -C2])

    intersection_point = np.linalg.solve(coeff_matrix, result_vector)

    return intersection_point

line1 = np.array([1, -1, -3]) # x - y - 3 = 0
line2 = np.array([1, 1, -1])  # x + y - 1 = 0

intersection_point = line_intersection(line1, line2)

print(f"The intersection point of the two lines is: {intersection_point}")


The intersection point of the two lines is: [ 2. -1.]


In [4]:
# A random 2D mesh
points = np.hstack((np.random.rand(100, 2), np.zeros((100, 1), float)))
mesh_2D = pv.PolyData(points).delaunay_2d()

# A random 2D line
# two_points = np.hstack((np.random.rand(2, 2), np.zeros((2, 1), float)))
x=np.array([random.random(), 0, 0])
y=np.array([0, random.random(), 0])
line=pv.Line(x, y)
line_coef=line_equation(x[:2],y[:2])
line.field_data["coef"]=line_coef

pl = pv.Plotter()
pl.add_mesh(mesh_2D,show_edges=True, point_size=15)
pl.add_points(points, color="red", point_size=5)
pl.add_mesh(line, show_edges=True, color="blue", line_width=8)
pl.camera_position = [(2, 2, -3), (0, 0, 1), (1, 0, 0)]
pl.show()

Widget(value="<iframe src='http://localhost:63248/index.html?ui=P_0x225c191dd10_0&reconnect=auto' style='width…

In [5]:
mesh_2D=mesh_2D.compute_cell_sizes(area=True, length=False, volume=False)

In [6]:
mesh_2D

PolyData (0x225c6c9cc40)
  N Cells:    183
  N Points:   100
  N Strips:   0
  X Bounds:   1.192e-02, 9.937e-01
  Y Bounds:   3.442e-02, 9.592e-01
  Z Bounds:   0.000e+00, 0.000e+00
  N Arrays:   1

In [7]:
line.field_data["coef"]

pyvista_ndarray([-0.75936266, -0.64434223,  0.48928943])

In [8]:
def directed_point_to_line_dist(p, line):
    x0, y0 = p
    A, B, C = line
    return (A*x0 + B*y0 + C) / np.sqrt(A**2 + B**2)

In [9]:
mesh_2D

PolyData (0x225c6c9cc40)
  N Cells:    183
  N Points:   100
  N Strips:   0
  X Bounds:   1.192e-02, 9.937e-01
  Y Bounds:   3.442e-02, 9.592e-01
  Z Bounds:   0.000e+00, 0.000e+00
  N Arrays:   1

In [10]:
mesh_2D.point_data["directed distance to line"]=[directed_point_to_line_dist(p, line_coef) for p in mesh_2D.points[:,:2]]

In [11]:
mesh_2D.point_data["directed distance to line"]

pyvista_ndarray([ 0.15852886, -0.22422421, -0.07877916, -0.34470303,
                 -0.44522403, -0.42528854,  0.16387958, -0.11460139,
                 -0.13143757,  0.02606832, -0.52032773, -0.4991277 ,
                 -0.32041391, -0.27330233, -0.27044519, -0.31630559,
                 -0.55053449, -0.6745497 , -0.18068536, -0.70543119,
                  0.29493141,  0.15073741, -0.16892354, -0.61284229,
                 -0.10636869,  0.40624533,  0.02673643,  0.06170719,
                 -0.56997165, -0.28385018, -0.62829629, -0.85562432,
                 -0.03210004, -0.17412838, -0.43566476, -0.09208174,
                 -0.37420525, -0.62283886,  0.18368403, -0.45782664,
                 -0.39827516, -0.09270292, -0.32609135, -0.63766761,
                 -0.1062335 , -0.16684216, -0.03851252, -0.26413191,
                  0.06438321, -0.12442932, -0.199898  , -0.25942982,
                  0.37501721, -0.34460214, -0.20659854,  0.17014455,
                 -0.48718183, -0.1

In [12]:
np.sum(mesh_2D.extract_cells(1).point_data["directed distance to line"] > 0)

0

In [13]:
mesh_2D.cell_data["num of vertex above"]=[np.sum(mesh_2D.extract_cells(i).point_data["directed distance to line"]>0) for i in range(mesh_2D.n_faces)]

In [14]:
mesh_2D.cell_data["num of vertex above"]

pyvista_ndarray([0, 0, 3, 3, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1,
                 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,
                 0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 1, 3, 1, 2, 1, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0,
                 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0,
                 2, 0, 0, 0, 3, 0, 0, 0, 3, 3, 2, 0, 0, 0, 0, 3, 2, 0, 0,
                 0, 0, 0, 0, 3, 3, 0, 1, 0, 0, 3, 3, 0, 0, 1, 0, 0, 3, 2,
                 3, 3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 1, 1, 0, 0,
                 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0])

In [15]:
intersect_id=[i for i in range(mesh_2D.n_faces) if mesh_2D.cell_data["num of vertex above"][i]>0 and mesh_2D.cell_data["num of vertex above"][i]<3]

In [16]:
above_id=[i for i in range(mesh_2D.n_faces) if mesh_2D.cell_data["num of vertex above"][i]==0]

In [17]:
under_id=[i for i in range(mesh_2D.n_faces) if mesh_2D.cell_data["num of vertex above"][i]==3]

In [18]:
pl = pv.Plotter()
pl.add_mesh(mesh_2D,show_edges=True, point_size=15, color='white')
pl.add_mesh(mesh_2D.extract_cells(intersect_id),show_edges=True, point_size=15, color='green')
pl.add_mesh(mesh_2D.extract_cells(above_id),show_edges=True, point_size=15, color='red')
pl.add_mesh(mesh_2D.extract_cells(under_id),show_edges=True, point_size=15, color='blue')
pl.add_points(points, color="red", point_size=5)
pl.add_mesh(line, show_edges=True, color="blue", line_width=8)
pl.camera_position = [(2, 2, -3), (0, 0, 1), (1, 0, 0)]
pl.show()

Widget(value="<iframe src='http://localhost:63248/index.html?ui=P_0x225c6cfda90_1&reconnect=auto' style='width…

In [19]:
mesh_2D.extract_cells(0).points

pyvista_ndarray([[0.57225125, 0.77309842, 0.        ],
                 [0.49472715, 0.68032965, 0.        ],
                 [0.45356611, 0.73179982, 0.        ]])

In [101]:
print([True,False,False]or[True, True, True])

[True, False, False]
